## Parsl Tutorial

Parsl is a native Python library that allows you to write functions that execute in parallel and tie them together with dependencies to create workflows. Parsl wraps Python functions as "Apps" using the **@App** decorator. Decorated functions can run in parallel when all their inputs are ready.

For more comprehensive documentation and examples, please refer our [documentation](parsl.readthedocs.io)

In [ ]:
import parsl
from parsl import *
# parsl.set_stream_logger() # <-- log everything to stdout
print(parsl.__version__)

### DataFlowKernal

Parsl's `DataFlowKernel` acts as an abstraction layer over any pool of execution resources (e.g., clusters, clouds, threads). 


We'll come back to the DataFlowKernel later in this tutorial. For now, we configure this example to use a pool of [threads](https://en.wikipedia.org/wiki/Thread_(computing). to facilitate local parallel exectuion. 

In [ ]:
# Let's create a pool of threads to execute our functions
workers = ThreadPoolExecutor(max_workers=4)

# We pass the workers to the DataFlowKernel which will execute our Apps over the workers.
dfk = DataFlowKernel(executors=[workers])

# 1. Apps


In Parsl an `app` is a piece of code that can be asynchronously executed on an execution resource (e.g., cloud, cluster, or local PC). Parsl provides support for pure Python apps and also command-line apps executed via Bash.

## Python Apps

As a first example let's define a simple Python function that returns the string 'Hello World!'. This function is made into a Parsl App using the **@App** decorator. The decorator specifies the type of App ('python'|'bash') and the `DataFlowKernel` object as arguments.

In [ ]:
@App('python', dfk)
def hello ():
    return 'Hello World!'

print (hello().result())

As can be seen above, Apps wrap standard Python function calls. As such, they can be passed arbitrary arguments and return standard Python objects. 

In [ ]:
@App('python', dfk)
def multiply (a, b):
    return a * b

print(multiply(5,9).result())

## Bash Apps

Parsl’s Bash app allows you to wrap execution of external applications from the command-line as you would in a Bash shell. It can also be used to execute Bash scripts directly. To define a Bash app the wrapped Python function must return the command-line string to be executed.

Parsl is able to capture stdout/stderr for debugging or as a first class data object in a workflow.


In [ ]:
@App('bash', dfk)
def echo_hello(stdout='echo-hello.stdout', stderr='echo-hello.stderr'):
    return 'echo "Hello World!"'

echo_hello().result()

with open('echo-hello.stdout', 'r') as f:
     print(f.read())

Often, Parsl Apps exchange data in the form of files. In order to orchestrate a dataflow it is important that Parsl is able to track the data that is passed into and out of an App. For this purpose Parsl Apps can define input and output files as follows.

In [ ]:
!echo "hello 1" > /tmp/hello1.txt
!echo "hello 2" > /tmp/hello2.txt
!echo "hello 3" > /tmp/hello3.txt

In [ ]:
@App('bash', dfk)
def cat(inputs=[], outputs=[]):
    return 'cat %s > %s' %(inputs[0], outputs[0]) 


release = cat(inputs=['/tmp/*hello*.txt'], outputs=['all_hellos.txt'])

with open(release.outputs[0].result(), 'r') as f:
     print(f.read())

# 2. Futures
When a Python function is invoked, the Python interpreter waits for the function to complete execution and returns the results. In case of long running functions it may not be desirable to wait for completion, instead it is often preferable that functions are asynchronous. Parsl provides such asynchronous behavior by returning a future in lieu of results. A future is essentially an object that allows us to track the status of an asynchronous task so that it may, in the future, be interrogated to find the status, results, exceptions, etc.

Parsl provides two types of futures: AppFutures and DataFutures. While related, these two types of futures enable subtly different workflow patterns, as we will see.



## AppFutures
AppFutures are the basic building block upon which Parsl scripts are built. Every invocation of a Parsl app returns an AppFuture which may be used to manage execution and control the workflow.

Here we show how AppFutures are used to wait for the result of a Python App.



In [ ]:
@App('python', dfk)
def hello ():
    import time
    time.sleep(5)
    return 'Hello World!'

app_future = hello()

# check if the app_future is resolved
print ('Done: %s' % app_future.done())

# print the result of the app_future
# this call will block and wait for the future to resolve
print ('Result: %s' % app_future.result())
print ('Done: %s' % app_future.done())

## DataFutures

While AppFutures represent the execution of an asynchronous app, the DataFuture represents the files it produces. Parsl’s dataflow model, in which data flows from one app to another via files, requires such a construct to enable apps to validate creation of required files and to subsequently resolve dependencies when input files are created. When invoking an app, Parsl requires that a list of output files be specified (using the outputs keyword argument). A DataFuture for each file is returned by the app when it is executed. Throughout execution of the app Parsl will monitor these files to 1) ensure they are created, and 2) pass them to any dependent apps.

In [ ]:
# This app echoes the input string to the first file specified in the
# outputs list
@App('bash', dfk)
def slowecho(message, outputs=[]):
    return 'sleep 5; echo %s &> {outputs[0]}' % (message)

# Call echo specifying the output file
hello = slowecho('Hello World!', outputs=['hello1.txt'])

# The AppFuture's outputs attribute is a list of DataFutures
print(hello.outputs)
# Also check the AppFuture
print ('Done: %s' % hello.done())

# Print the contents of the output DataFuture when complete
with open(hello.outputs[0].result(), 'r') as f:
     print(f.read())
        
# Now that this is complete, check the DataFutures again, and the Appfuture
print(hello.outputs) # should say done, but current bug makes it saying pending even when it is done
print ('Done: %s' % hello.done())

# 3. Data Management

Parsl is designed to enable implementation of dataflow patterns. These patterns enable workflows to be defined in which the data passed between apps manages the flow of execution. Dataflow programming models are popular as they can cleanly express, via implicit parallelism, the concurrency needed by many applications in a simple and intuitive way.

## Files

Parsl’s file abstraction abstracts local access to a file. It therefore requires only the file path to be defined. Irrespective of where the script, or its apps are executed, Parsl uses this abstraction to access that file. When referencing a Parsl file in an app, Parsl maps the object to the appropriate access path.



In [ ]:
from parsl.data_provider.files import File

@App('bash', dfk)
def cat(inputs=[], outputs=[]):
     return 'cat %s &> %s' % (inputs[0], outputs[0])

# create a test file
open('cat-in.txt', 'w').write('Hello World!\n')

# create the Parsl files
parsl_infile = File("cat-in.txt")
parsl_outfile = File("cat-out.txt")

# call the cat app with the Parsl file
cat_future = cat(inputs=[parsl_infile], outputs=[parsl_outfile])

# read what was redirected to stdout
with open(cat_future.outputs[0].result(), 'r') as f:
     print(f.read())

# 4.  Composing a workflow

Now that we understand all the building blocks, we can create workflows with Parsl. Unlike other workflow systems, Parsl creates implicit workflows based on the passing of control or data between Apps. The flexibility of this model allows for the creation of a wide range of workflows from sequential through to complex nested, parallel workflows. As we will see below, a range of workflows can be created by passing AppFutures and DataFutures between Apps.


## Sequential workflow

Simple sequential or procedural workflows can be created by passing an AppFuture from one task to another. The following example shows one such workflow, which first generates a random number and then writes it to a file. 


In [ ]:
# Generate a random number
@App('python', dfk)
def generate(limit):
      from random import randint
      """Generate a random integer (between 1 and limit) and return it"""
      return randint(1,limit)

# write a message to a file
@App('bash', dfk)
def save(message, outputs=[]):
      return 'echo %s &> {outputs[0]}' % (message)

message = generate(10)
print('random number =',message.result())

saved = save(message, outputs=['output.txt'])

with open(saved.outputs[0].result(), 'r') as f:
      print(f.read())

## Parallel workflow

The most common way that Parsl Apps are executed in parallel is via looping. The following example shows how a simple loop can be used to create many random numbers in parallel.


In [ ]:
@App('python', dfk)
def generate(limit):
    from random import randint
    """Generate a random integer (between 1 and limit) and return it"""
    return randint(1,limit)

rand_nums = []
for i in range(5):
    rand_nums.append(generate(10))

# wait for all apps to finish and collect the results
outputs = [i.result() for i in rand_nums]

print(outputs)

## Parallel dataflow

Parallel dataflows can be developed by passing data between Apps. In this example we create a set of files, each with a random number, we then concatenate these files into a single file and compute the sum of all numbers in that file. In the first two Apps files are exchanged. The final App returns the sum as a Python integer.



In [ ]:
@App('bash', dfk)
def generate(outputs=[]):
    # Generate random number between 5 and 10
    return "echo $(( (RANDOM % 6 ) + 5 )) &> {outputs[0]}"

@App('bash', dfk)
def concat(inputs=[], outputs=[], stdout="stdout.txt", stderr='stderr.txt'):
    return "cat {0} > {1}".format(" ".join(inputs), outputs[0])

@App('python', dfk)
def total(inputs=[]):
    total = 0
    with open(inputs[0], 'r') as f:
        for l in f:
            total += int(l)
    return total

# create 5 files with random numbers
output_files = []
for i in range (5):
     output_files.append(generate(outputs=['random-%s.txt' % i]))

# concatenate the files into a single file
cc = concat(inputs=[i.outputs[0] for i in output_files], outputs=["all.txt"])

# calculate the sum of the random numbers
total = total(inputs=[cc.outputs[0]])

print (total.result())

# Execution and configuration

Parsl is designed to support arbitrary execution providers (e.g., PCs, clusters, supercomputers) and execution models (e.g., threads, pilot jobs, etc.). Parsl scripts are independent of execution provider or executor. Instead, the configuration used to run the script tells Parsl how to execute apps on the desired environment. Parsl provides a high level abstraction, called a Block, for describing the resource configuration for a particular app or script.

Information about the different execution providers and executors supported is included in the [Parsl documentation](https://parsl.readthedocs.io/en/latest/userguide/execution.html)


## Local execution with threads

As we saw above, we can define an `executor` that uses a thread pool locally. This is a good way to parallelize execution on a local PC.  There are two ways to configure Parsl for thread execution. The first is by passing the executor object to Parsl's DataFlowKernel. The second is by creating a configuration object to create the DataFlowKernel. The configuration approach is typically used in more complex environments and has many more options required.

In [ ]:
# create the dfk using a threadpool executor
workers = ThreadPoolExecutor(max_workers=4)
dfk = DataFlowKernel(executors=[workers])

# define a configuration for using threads
local_config = {
    "sites" : [
        { "site" : "Local_Threads",
          "auth" : { "channel" : None },
          "execution" : {
              "executor" : "threads",
              "provider" : None,
              "max_workers" : 4
          }
        }],
    "globals" : {"lazyErrors" : True}
}
dfk = DataFlowKernel(config=local_config)

## Local execution with pilot jobs

We can also define a configuration that uses IPythonParallel as the executor. In this mode, pilot jobs are used to manage the submission. Parsl creates an IPythonParallel controller to manage execution and deploys 1 or more IPythonParallel engines (workers) to execute workload. The following config will instantiate this infrastructure locally, it can be trivially extended to include a remote provider (e.g., Cori, Theta, etc.) for execution. 

In [ ]:
ipp_config = {
    "sites" : [{
        "site" : "Local_IPP",
        "auth" : {
            "channel" : "local"
        },
        "execution" : {
            "executor" : "ipp",
            "provider" : "local",
            "script_dir" : ".scripts",
            "scriptDir" : ".scripts",
            "block" : {
                "nodes" : 1,
                "taskBlocks" : 1,
                "walltime" : "00:05:00",
                "initBlocks" : 1,
                "minBlocks" : 0,
                "maxBlocks" : 1,
                "scriptDir" : ".",
                "options" : {
                    "partition" : "debug"
                }
            }
        }
    }],
    "globals" : { "lazyErrors" : True },
    "controller" : { "publicIp" : "*" }
}

## Running a workflow using a configuration

We can now run the same workflow using either of the two configurations defined above. Change which config is used to instantiate the DFK to see the same workflow executed with different models. 

In [ ]:
#dfk = DataFlowKernel(config=local_config)
dfk = DataFlowKernel(config=ipp_config)

@App('bash', dfk)
def generate(outputs=[]):
    # Generate random number between 5 and 10
    return "echo $(( ( RANDOM % 6 ) + 5 )) &> {outputs[0]}"

@App('bash', dfk)
def concat(inputs=[], outputs=[], stdout="stdout.txt", stderr='stderr.txt'):
    return "cat {0} > {1}".format(" ".join(inputs), outputs[0])

@App('python', dfk)
def total(inputs=[]):
    total = 0
    with open(inputs[0], 'r') as f:
        for l in f:
            total += int(l)
    return total

# create 5 files with random numbers
output_files = []
for i in range (5):
     output_files.append(generate(outputs=['random-%s.txt' % i]))

# concatenate the files into a single file
cc = concat(inputs=[i.outputs[0] for i in output_files], outputs=["all.txt"])

# calculate the sum of the random numbers
total = total(inputs=[cc.outputs[0]])

print (total.result())

dfk.cleanup()